# Gene knockouts

FBA is classically very good at predicting the effect of knocking out genes
in an organism. Here we demonstrate the ways of using the FBA to examine
knockouts in COBREXA.

As usual, we need packages and models:

In [1]:
using COBREXA

download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

import JSONFBCModels
import GLPK

model = load_model("e_coli_core.json")

[ Info: using cached `e_coli_core.json'


JSONFBCModels.JSONFBCModel(#= 95 reactions, 72 metabolites =#)

## Single gene knockouts

Function `gene_knockouts` is a convenience wrapper for FBA that
computes and optimizes the knockout biomass productions for all genes:

In [2]:
ko_objective_values = gene_knockouts(model, optimizer = GLPK.Optimizer)

ko_dict = Dict(ko_objective_values)


ko_dict["b3919"]

ko_dict["b3738"]

0.37422987493310866

From the result, we can see e.g. how many genes are critical:

In [3]:
critical = count(isnothing, values(ko_dict))

2

## Multiple gene knockouts

By default, `gene_knockouts` simply computes all gene knockouts. To
examine multi-gene knockouts, we specify them manually as an array of tuples:

In [4]:
some_double_knockouts = gene_knockouts(
    model,
    [("b3919", "b3738"), ("b0118", "b0720")],
    optimizer = GLPK.Optimizer,
)

2-element Vector{Pair{Tuple{String, String}, Float64}}:
 ("b3919", "b3738") => 0.13475540327383498
 ("b0118", "b0720") => -1.0222950287826093e-15

With the array processing functionality of Julia it is quite straightforward
to generate the tuples for various specifications of knockout sets; for
example here we specify all double knockout where the second knocked-out gene
is `b3919`:

In [5]:
knockouts_with_b3919 = gene_knockouts(
    model,
    tuple.(keys(ko_dict), "b3919"),
    optimizer = GLPK.Optimizer,
    settings = [silence],
)

137-element Vector{Pair{Tuple{String, String}}}:
 ("b3236", "b3919") => 0.7040369478590305
 ("b1621", "b3919") => 0.70403694785902
 ("b1241", "b3919") => 0.70403694785902
 ("b2276", "b3919") => nothing
 ("b3925", "b3919") => 0.70403694785902
 ("b0979", "b3919") => 0.70403694785902
 ("b2296", "b3919") => 0.70403694785902
 ("b4232", "b3919") => 0.70403694785902
 ("b2282", "b3919") => nothing
 ("b2283", "b3919") => nothing
                    ⋮
 ("b0903", "b3919") => 0.70403694785902
 ("b1723", "b3919") => 0.70403694785902
 ("b0810", "b3919") => 0.70403694785902
 ("b0356", "b3919") => 0.70403694785902
 ("b2277", "b3919") => nothing
 ("b1276", "b3919") => 0.70403694785902
 ("b4153", "b3919") => 0.70403694785902
 ("b2279", "b3919") => nothing
 ("b0726", "b3919") => 0.7040369478590158

Now, how many genes are critical given `b3919` is already missing?

In [6]:
critical_without_b3919 = count(isnothing, last.(knockouts_with_b3919))

23

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*